# 002 - Window Functions

```SQL
DROP TABLE IF EXISTS baby_names;

CREATE TABLE baby_names (
    Gender VARCHAR(10),
    Name VARCHAR(50),
    Total INT
);

INSERT INTO baby_names (Gender, Name, Total) VALUES
('Girl', 'Ava', 95),
('Girl', 'Emma', 106),
('Boy', 'Ethan', 115),
('Girl', 'Isabella', 100),
('Boy', 'Jacob', 101),
('Boy', 'Liam', 84),
('Boy', 'Logan', 73),
('Boy', 'Noah', 120),
('Girl', 'Olivia', 100),
('Girl', 'Sophia', 88);

SELECT * FROM baby_names;

-- 1. View the table
SELECT *
FROM baby_names;

-- 2. Order by popularity

SELECT *
FROM baby_names
ORDER BY Total DESC;

-- 3. add a popularity column

SELECT Gender,
       Name,
       Total,
       ROW_NUMBER() OVER(ORDER BY Total DESC) AS Popularity
FROM baby_names;

-- 4. try different column

SELECT Gender, Name, Total,
       ROW_NUMBER() OVER(ORDER BY Total DESC) AS Popularity,
       RANK() OVER(ORDER BY Total DESC) AS Popularity_R,
       DENSE_RANK() OVER(ORDER BY Total DESC) AS Popularity_DR
FROM baby_names;

-- 5. Try different windows

SELECT Gender, Name, Total,
       ROW_NUMBER() OVER(PARTITION BY Gender ORDER BY Total DESC) AS Popularity
FROM baby_names;

-- 6. What are the top 3 most popular names for each gender?

SELECT * FROM
(SELECT Gender, Name, Total,
       ROW_NUMBER() OVER(PARTITION BY Gender ORDER BY Total DESC) AS Popularity
FROM baby_names) AS pop
WHERE Popularity <= 3;
```

In [1]:
import pandas as pd

data = {
    'Gender': ['Girl', 'Girl', 'Boy', 'Girl', 'Boy', 'Boy', 'Boy', 'Boy', 'Girl', 'Girl'],
    'Name': ['Ava', 'Emma', 'Ethan', 'Isabella', 'Jacob', 'Liam', 'Logan', 'Noah', 'Olivia', 'Sophia'],
    'Total': [95, 106, 115, 100, 101, 84, 73, 120, 100, 88]
}

df = pd.DataFrame(data)

In [2]:
# Primero ordenamos por Total descendente para que la visualización sea clara
df = df.sort_values('Total', ascending=False)

# ROW_NUMBER -> method='first' (asigna números únicos según el orden de aparición)
df['Popularity'] = df['Total'].rank(ascending=False, method='first').astype(int)

# RANK -> method='min' (si hay empate, ambos tienen el mismo número y salta el siguiente)
df['Popularity_R'] = df['Total'].rank(ascending=False, method='min').astype(int)

# DENSE_RANK -> method='dense' (si hay empate, no salta el siguiente número)
df['Popularity_DR'] = df['Total'].rank(ascending=False, method='dense').astype(int)

print(df)

  Gender      Name  Total  Popularity  Popularity_R  Popularity_DR
7    Boy      Noah    120           1             1              1
2    Boy     Ethan    115           2             2              2
1   Girl      Emma    106           3             3              3
4    Boy     Jacob    101           4             4              4
8   Girl    Olivia    100           5             5              5
3   Girl  Isabella    100           6             5              5
0   Girl       Ava     95           7             7              6
9   Girl    Sophia     88           8             8              7
5    Boy      Liam     84           9             9              8
6    Boy     Logan     73          10            10              9


In [3]:
# PARTITION BY Gender ORDER BY Total DESC
df['Popularity_by_Gender'] = df.groupby('Gender')['Total'].rank(ascending=False, method='first').astype(int)

In [5]:
# 1. Calculamos el ranking dentro del grupo
df['Popularity'] = df.groupby('Gender')['Total'].rank(ascending=False, method='first').astype(int)

# 2. Filtramos (el equivalente al WHERE Popularity <= 3)
top_3_names = df[df['Popularity'] <= 3].sort_values(['Gender', 'Popularity'])

print(top_3_names)

  Gender      Name  Total  Popularity  Popularity_R  Popularity_DR  \
7    Boy      Noah    120           1             1              1   
2    Boy     Ethan    115           2             2              2   
4    Boy     Jacob    101           3             4              4   
1   Girl      Emma    106           1             3              3   
8   Girl    Olivia    100           2             5              5   
3   Girl  Isabella    100           3             5              5   

   Popularity_by_Gender  
7                     1  
2                     2  
4                     3  
1                     1  
8                     2  
3                     3  
